In [1]:
import datetime
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import pyodbc
import multiprocessing
from keras.models import load_model
from joblib import Parallel, delayed
import xml.etree.ElementTree as xml
def all_features(data):
    data=data.groupby(['DiscountCardId'])['GoodsGroup1Id'].agg(lambda x: list(x))
    data=data.reset_index(drop=False) 
    data.rename(columns={'GoodsGroup1Id':'goods'},inplace=True)
    data['goods']=data['goods'].apply(lambda x:np.squeeze(pad_sequences([x],maxlen=400,dtype='int32')))
    return data
def preprocessing(data):
#    goods=pd.read_csv(r'D:\Projects\Goods_prediction\real_data\goods2id.csv',sep=';',names=['GoodsGroup2Id','goodsname'])
#    goods['GoodsGroup2Id']=goods['GoodsGroup2Id'].astype(int)
#    data=data.merge(goods.reset_index(drop=False),how='left',on='GoodsGroup2Id')
#    data=data.drop(['GoodsGroup2Id','goodsname'],axis=1)
#    data=data.rename(columns={'index':'GoodsGroup2Id'})
#    data['GoodsGroup2Id']=data['GoodsGroup2Id'].astype(np.int32)
    data=all_features(data)
#    model.predict(np.asarray([x for x in data['goods']],dtype=np.int16))
    return data
def model_prediction(data):
    pr=model.predict(np.asarray([x for x in data],dtype=np.int16),batch_size=4000)
    return pr
np.set_printoptions(suppress=True)
def create_xml(Discount,pr):
    root=xml.Element("DiscountCardId")
    root.text=str(Discount)
    for goodsid,probability in zip(goods['GoodsGroup2Id'].values,pr):
        temp=xml.SubElement(root,"GoodsGroup2Id")
 #       temp.text=str(goodsid)
        temp3=xml.SubElement(temp,"Value")
        temp3.text=str(goodsid)
        temp2=xml.SubElement(temp,"Probability")
        temp2.text=str(round(float(probability),4))
        
        
    tree=xml.ElementTree(root)
    return tree
def weeksdate(date):
    for i,(a,b) in enumerate(zip(wk['DateWeekBegin'].values,wk['DateWeekEnd'].values)):
        if a<=date and date<=b:
            return wk['WeekNumber'][i],wk['Years'][i]
def weeksdate(date):
    for i in range(len(wk)):
        if wk['DateWeekBegin'][i]<=date and date<=wk['DateWeekEnd'][i]:
            return wk['DateWeekBegin'][i],wk['DateWeekEnd'][i],wk['WeekNumber'][i],wk['Years'][i]

Using TensorFlow backend.


In [2]:
%%time
model=load_model(r'D:\Projects\Goods_prediction\real_data\model_rnn_mDense_2l.hdf5')
database_data="DRIVER={SQL Server}; SERVER=; DATABASE=; Trusted_Connection=yes;"
conn=pyodbc.connect(database_data)

from sqlalchemy import create_engine
import urllib
quoted = urllib.parse.quote_plus("DRIVER={SQL Server}; SERVER=; DATABASE=Discount; Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

base_date=pd.Timestamp(2018,9,3)
goods=pd.read_csv(r'D:\Projects\Goods_prediction\real_data\goods2id.csv',sep=';',names=['GoodsGroup2Id','goodsname'])
goods['GoodsGroup2Id']=goods['GoodsGroup2Id'].astype(int)
wk=pd.read_csv(r'D:\Projects\Goods_prediction\real_data\Weeks.csv',sep=';')
wk['DateWeekBegin']=pd.to_datetime(wk['DateWeekBegin'],format='%d.%m.%Y')
wk['DateWeekEnd']=pd.to_datetime(wk['DateWeekEnd'],format='%d.%m.%Y')
inputs=[51,52,53,54,55]
def result(i):
    data_period=base_date+pd.DateOffset(weeks=i)
    start_data=data_period-pd.DateOffset(months=3)
    label_period=base_date+pd.DateOffset(weeks=i+1)
    sql=f"""
***
    """
    data=pd.read_sql(sql,conn)
    data=preprocessing(data)
    pr=model_prediction(data['goods'].values)
    weekstart,weekend,weekyear,year=weeksdate(data_period)
    itog=[]
    for i,discountid in enumerate(data['DiscountCardId']):
        itog.append([discountid,xml.tostring(create_xml(data['DiscountCardId'][i],pr[i]).getroot())])
#        if i%10000==0:
#            print("10000 completed")
    result=pd.DataFrame(itog,columns=['DiscountCardId','PredictionNN'])
    result['TimeStart']=weekstart
    result['TimeEnd']=weekend
    result['WeekNumber']=weekyear
    result['WeekYear']=year
    result['PredictionNN']=result['PredictionNN'].astype(str)
    result['PredictionNN']=result['PredictionNN'].str.replace("b\'","").str.replace("\'","")
    result.to_sql(index=False,name='PredictionNN',schema='Prediction',con=engine,if_exists='append')
    print('one week is ready')
    return len(result)
# for i in inputs:
#     result(i)
result(57)

W1008 20:26:28.339768 11896 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1008 20:26:28.352735 11896 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1008 20:26:28.408612 11896 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1008 20:26:28.409610 11896 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:1

one week is ready
Wall time: 1h 43min 3s


222133

W0930 16:10:58.233216 17760 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 16:10:58.256188 17760 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0930 16:10:59.236805 17760 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0930 16:10:59.236805 17760 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:1

Wall time: 29min 46s


10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed
10000 completed


AssertionError: 1 columns passed, passed data had 2 columns

(222133, 788)

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n\r\n<html>\r\n\r\n\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>\r\n<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7" />\r\n<script language="javascript" src="/var.js" ></script>\r\n<script>loadJS("/common.js");</script>\r\n\r\n<script>InitLoad();</script>\r\n<script>\r\n\r\n\tif( g_oemname==\'etisalat\' && g_firstlogin==1){\t\t\r\n\t\t\r\n\t\talert(GL("err_etisalat_msg"));\r\n\t\t location.href="maintenance.htm";\r\n\t}\t \r\n\r\n\t\t \r\n\tloadJS("/D-Link.js");\r\n\tloadJS("/index.js");\r\n\tloadJS("/ptzctrl.js");\r\n\tloadJS("/ptz_dlink.js");\r\n</script>\r\n\r\n<script>\r\n//<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7" />\r\n//<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"\r\n //  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\nif(g_isSupportEPTZ == 1)\r\n{\r\n  loadJS("/uvumi/mootools-for-cro